# Data Scientist Nano Degree

## Capstone Project

Michael Reinhard   
August 28, 2019

### Project Overview

In this project I have attempted to train a model which distinguishes between positive and negative reviews for food products purchased on Amazon. It uses a publically available data set from the Kaggle.com website. 

There is a lot of information available to retailers from consumers that is in text form or unstructured form. It is valuable information but difficult to analyze. Structured information can be obtained, say, through surveys adn the like, but they suffer from response bias or are costly to effectuate. Text data, such as that available in online reviews is a valuable and, for the most part, free source of information, but it is much more difficult than structured data to analyze. 

This project seeks to demonstrate the viability of analyzing such data. The Amazon Food Reviews data set consists of over 600,000 reviews by customers and a rating of from 1 to 5 stars by those same customers. Thus, we have both the unstructured text data of the customer reviews and the structured, numerical data of those same customers' numerical ratings of the food products. This pairing of text and numerical data invites the use of supervised learning algorithms to see if we can correctly predict the star rating from the reviewer's words. 

We can use a model trained on this data to build an app that classifies raw text as a positive or negative review, thus demonstrating the principle that raw text can be used as quantitative data. 

### Metrics

I went back and forth on this. It dependes on how you define a positive review. If you choose a cut off threshold of 4, meaning that reveiws of 4 and 5 stars will be classified as positive and those below negative, you have a skewed data set with 78% positive reviews. If you included only 5 star reviews in your definition of positive you get a much more balanced data set--about 63% positive--but you miss part of what is meant by positive review. There are a lot of 4 star reviews that are very positive but have one or two quibbles. Also, the text of 4 star and 5 start reviews might be harder to distinguish than, say, 4 star and 3 star reviews. 

The problem with an unbalanced data set it that in training the model the model can get 78% accuracy just by guessing positive all the time. With accuracy the default scoring method that could cause a problem. 

The paradox of high accuracy models could be address by changing the scoring method to something else that maximizes not only accuracy but all kind of error, both cases that are mistakenly classified as positive and cases that are mistakenly classified as negative. One such scoring method is 'negative log loss', which I used in most of my experimentation in the notebook. 

However, I am somewhat embarrassed to say that I could not find a way to institute negative-log-loss scoring in the script. Since the script is what is going to be displayed to the public I decided to let my scoring method choice dictate my positive-negative split choice. So I made the split between 4 and 5. This left me with a more balanced data set that could be handled well with accuracy based models. 

### Data Exploration

There are some other variables in the data set such as an average rating for helpfulness of a review given by the other users but I chose to focus only on the text. This allowed me to focus on NLP. 

One thing that should be noted is that there are about 20% by my estimate of the food reviews that deal with pet food. I don't know what effect that has on the results or if the pet food reviews differ from their human counter-parts but it is a subject for further investigation. 

### Exploratory Visualization

![Distribution](newplot.png)

![title](newplot_1.png)

The first figure shows the counts of reviews in each category. We can see that over half the data set is composed of 5 star reviews. The next most numerous category is 4 star reviews which together make up over 78% of the reviews. 

The second fugure shows the proportions of negative and positive reviews after recoding the 5 star reviews to positive and the rest to negative. This makes for a relatively balanced data set, meaning the model won't be tempted to maximize accuracy by just classify everything as belonging to the most common category. 

### Algorithms and Techniques

I tried three algorithms on this data: Multinomial Naive Baysian analysis, Support Vector Classification, and the Random Forest Classifier. I ultimately decided to proceed with the Support Vector Classifier. Its performance was superior to the Naive Bayes classifier and it ran a lot faster than the Random Forest Classifier. 

I tried multiple parameters with the support vector classifier. Some initial experimentation led me to believe that the default "rbf" or Radial Basic Function outperformed or performed at least as well as the linear, poly, or sigmoid kernels so I decided to not tune that parameter. Since the gamma is part of the rbf kernel I had to tune that and I tuned the C parameter. Though very different in magnitude with gamma tending to be between 1 and 0 and C measured in orders of magnitude, I found that they tended to move in the same direction. In tuning the value of gamma tended to be large so as to allow the model to make a lot of exceptions or grant a lot of influence to individual observations. The C values also tended to be large, again consistent with the complicated decision surface that text data engenders.  

### Benchmark

I instituted a benchmark by instantiating a dummy model, i.e., one that always guesses the most common category, but I found it to be of limited usefulness for this project as the implications of the dummy model are fairly straightforward in a binary classification model. The confusion matrix resulting from the implementation of the dummy model is seen below.

![title](download.png)

### Data Preprocessing

Data preprocessing is expecially important for a model that deals with text data. 

I first wrote a function that tokenized the text. I decided to use spaCy for this task instead of the nltk package. I found that it was a lot faster then the nltk package since, as the capital 'C' in the name reminds us, it is written to run C underneath a python wrapper. The function removes all web addresses using a regular expression. I wrote in options to remove stopwords and lemmatise the text, both of which options I ended up employing in the final model. 

I also wrote a brief function to standardize the data, turning the counts into standard deviations for a 0 mean, but I ended up not using it. Instead, I decided to use the dimension ruduction routine Singular Value Decomposition to reduce the dimensionality of the data. Since the data would be standardized in the course of implementing Singular Value Decomposition. 

Also, I implemented TfIdf--the Term frequency Inverse document frequency algorithm. This algorithm takes the frequency of terms that are infrequent in the corpus as an indicator of importance. It too has parameters, namely, the minimum and maximum proportion of the documents a term has to appear in to be considered by the algorithm, which I tuned. There are also a maximum number of features, a n-gram range (i.e., whether you want to consider each word in isolation or in conjunction with 1 or 2 of the words around it) which I also tuned. 

Finally, on the vectorized data, I instituted one more preprocessing step: singular value decomposition. This results in the reduction of the dimensionality in the data. The number of dimensions you want to retain is also a parameter that is tunable. 

### Implementation

I implemented a Randomized Grid Search for tuning these parameters instead of a standard GridSearch. I have heard of studies that argue that randomized grid search is as effective as Grid Search (which takes every possible combination of parameters rather than a sample of them). But the main reason I used randomized grid search was practicality. I just had too many parameters to search through every possible combination. Also, with the time it takes to process text data anything but a randomized search would have been impractical. 

I also compromised my search by using samples of the data rather than the whole data set for tuning the parameters. First I would try a search on 1000 samples, moving up to 10,000 and, at last, 100,000 before trying anything on the full data set of over 600,000 reviews.

### Refinement

Looking back I think I made a mistake in my use of randomized grid search. When I found that the extreme value of one of my parameters was included in the best parameters I would add a more extreme value next to it and, simultaniously, remove a value at the other extreme. Looking back, I think this may have introduced a bias into my process. What if, as seems likely, the extreme value of a parameter made it into the set of best parameters because the next most exteme value was not sampled? As I am simultaneously removing values at the other extreme maybe I was inadvertantly biasing the sample space, excluding values that could have, in other combinations, been included in the best parameters themselves, and introducing values into the parameter sample space that had no business being there. In future iterations of this project I will strictly adhere to the randomized sampling procedure.  

### Model Evaluation and Validation

The tuning process eventually produced this set of parameters as the best:

'tfidf__ngram_range': (1, 2)
'tfidf__min_df': 1e-05
'tfidf__max_features': 1300
'tfidf__max_df': 0.7
'reduce_dim__n_components': 140
'clf__gamma': 0.01
'clf__C': 10000 

The model finished with a log loss of 0.49408760389845235, as compared to a log loss of 12.44 for the dummy model. 

The accuracy of the model is 0.79, meaning that it correctly almost 80% of the reviews, as opposed to the 63% possible if one had merely guessed the most common category for each case. 

Finally, as can be seen from the percision and recall numbers as well as the f1 score, the model is fairly even between false positives and false negatives. In future versions of the model this can be adjusted. For instance, if one is using the model to decide which customers to initiate contact with one could train the model to maximize recall so as to not let any customers that are favorably inclined to your product go uncontacted.  

              precision    recall  f1-score   support

           0       0.75      0.62      0.68       909
           1       0.80      0.88      0.84      1591

    accuracy                           0.79      2500
   